In [11]:

import socket
import os

# lasciando il campo vuoto sarebbe la stessa cosa (localhost)
SERVER_ADDRESS = '127.0.0.1'

# Numero di porta, deve essere >1024 perchè le altre sono riservate.)
SERVER_PORT = 22224

# Crea la socket
s = socket.socket()

# Opzionale: permette di riavviare subito il codice, 
# altrimenti bisognerebbe aspettare 2-4 minuti prima di poter riutilizzare(bindare) la stessa porta
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Associa indirizzo e porta.  Nota che  l'argumento è una tupla:
s.bind((SERVER_ADDRESS, SERVER_PORT))

# Imposta quante connessioni pendenti possono essere accodate
s.listen(5)

print("Server in ascolto su %s. Termina il server con Ctrl-C" % str((SERVER_ADDRESS, SERVER_PORT)))

# Abbiamo creato un endpoint di ascolto dal quale accettare connessioni in entrata
# Il loop accetterà una connessione alla volta e la servirà finchè il client non si disconnette
# dopodichè il server ne accetterà di nuove tra quelle in coda

while True:
    c, addr = s.accept()
    print("\nConnessione ricevuta da %s" % str(addr))

    while True:
        request = c.recv(1024)
        request=request.decode()
        request=request.strip()
        print ("[*] Received: %s" % request)
        # send back a packet
        if request == "#list":
            print(os.listdir('./fileserver'))
            c.send(str(os.listdir('./fileserver')).encode())
        elif request.find('#get') != -1:
            param = request.split('/')
            nomefile=str(param[1])
            url = './fileserver/'+nomefile
            if (os.path.isfile(url)):
                c.send("OK".encode())
                f = open(url, "r")
                testo = f.read()
                print ("Contenuto del file " + url + " richiesto=" + testo)
                c.send(testo.encode())
                f.close()
            else:
                c.send("KO".encode())
            print (url)
        elif request.find('#put') != -1:
            param = request.split('/')
            nomefile = str(param[1])
            url = './fileserver/' + nomefile
            print (url)
            f = open(url, "w")
            f.write(param[2])
            f.close()
            print ("Invio esito della scrittura su: " + param[1] + " contenuto:  " + param[2])
            c.send("OK".encode())
            
        elif request == "#close":
            #print response
            fine = 0
            print ("Termino la connessione")
            c.send("closed".encode())
            fine=1
        print (c.getpeername())

    c.close()

Server in ascolto su ('127.0.0.1', 22224). Termina il server con Ctrl-C

Connessione ricevuta da ('127.0.0.1', 50858)
[*] Received: #list
['.DS_Store', 'sentimenti_neg.txt', 'sentimenti_pos.txt']
('127.0.0.1', 50858)
[*] Received: #get /sentimenti_pos.txt
Contenuto del file ./fileserver/sentimenti_pos.txt richiesto=agiato,esuberante,orgoglioso,rinvigorito,affascinato,euforico,ottimista,riposato,rassicurato,felice,pacato,sbalordito,allegro,fiducioso, sereno, appassionato,gioioso,sicuro,assorto,grato,creativo,soddisfatto, attento,impegnato,meravigliato,sollevato, calmo,armonia,simpatia,sorpreso,coinvolto, pace,potente,spensierato, commosso,incuriosito,pronto,stimolato,
compiaciuto,interessato,rapito,stupito, concentrato,ispirato,realizzato,tranquillo, divertito,leggero,riconoscente,vitale,emozionato,libero,rilassato,vivace, entusiasta,lucido,rincuorato,zelante, appagato, raggiante,estasiato
./fileserver/sentimenti_pos.txt
('127.0.0.1', 50858)
[*] Received: #put /appunti.txt/prova prova p

TypeError: a bytes-like object is required, not 'str'